In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [ ]:
adata = sc.read_h5ad("../data/adata/timecourse.h5ad")

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
# Coordinates of the gates
gates = {
    "Top": {
        "edges": [
            [0.15, 0.5],
            [0.6, 0.7],
            [0.8, 0.7],
            [0.8, 1.03],
            [0.15, 1.03],
        ],
        "label_position": {"x": 0.16, "y": 0.9},
        "fill": "#3A9AB244",
        "stroke": "#3A9AB2",
    },
    "Crypt": {
        "edges": [
            [0.15, 0.48],
            [0.6, 0.68],
            [0.8, 0.68],
            [0.8, 0.25],
            [0.2, 0],
            [0.15, 0],
        ],
        "label_position": {"x": 0.16, "y": 0.05},
        "fill": "#F11B0044",
        "stroke": "#F11B00",
    },
    "Muscularis": {
        "edges": [[0.22, 0], [0.8, 0.23], [6, 0.23], [6, 0], [0.22, 0]],
        "label_position": {"x": 0.6, "y": 0.05},
        "fill": "#BDC88155",
        "stroke": "#BDC881",
    },
}

In [ ]:
# Custom biexponential transformation. Maybe not needed for IF data
def transformation(x, a=0.1, b=0.1, c=0.5, d=2.5, f=4, w=1):
    x = np.array(x)
    return a * np.exp(b * ((x - w))) - c * np.exp(-d * (x - w)) + f


def classify_cells(adata, gates, transformation=transformation):
    """
    Classify cells based on the gates.
    """
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    import geopandas as gpd

    adata.obs["epithelial_distance_transformed"] = transformation(
        adata.obs["epithelial_distance"]
    )
    adata.obs["gate"] = False

    print("Creating polygons")
    polygons = {}
    for gate in gates:
        # Apply transformation to x values
        points = [
            [transformation(element[0])] + element[1:]
            for element in gates[gate]["edges"]
        ]
        polygons[gate] = Polygon(points)
    polygons = gpd.GeoSeries(polygons)
    gpd_poly = gpd.GeoDataFrame({"gates": polygons}, geometry="gates")

    print("Creating cells")
    cells = gpd.GeoSeries.from_xy(
        adata.obs["epithelial_distance_transformed"], adata.obs["crypt_villi_axis"]
    )
    gpd_cells = gpd.GeoDataFrame({"cells": cells}, geometry="cells")

    print("Joining cells and polygons")
    result = gpd.sjoin(
        gpd_cells,
        gpd_poly,
        how="left",
    )
    return result

# Assign all cells

In [ ]:
classification = classify_cells(adata, gates)
classification

In [ ]:
adata.obs["gate"] = classification["index_right"]
adata

In [ ]:
def make_name(gate, cell):
    if cell == "Cd8_T-Cell_P14":
        if gate == "Top":
            return "P14 top"
        elif gate == "Crypt":
            return "P14 crypt"
        elif gate == "Muscularis":
            return "P14 muscularis"
        else:
            return "P14 undeterminded"
    else:
        return cell


adata.obs["Subtype_gate"] = [
    make_name(gate, cell) for gate, cell in zip(adata.obs["gate"], adata.obs["Subtype"])
]
adata = adata[~(adata.obs["Subtype_gate"] == "P14 undeterminded")]
adata.obs["Subtype_gate"] = adata.obs["Subtype_gate"].astype("category")
adata.obs["Subtype_gate"]
adata

In [ ]:
fig, ax = plt.subplots()
for g in adata.obs["gate"].unique():
    ax.scatter(
        adata[adata.obs["gate"] == g].obs[["epithelial_distance_transformed"]],
        adata[adata.obs["gate"] == g].obs["crypt_villi_axis"],
        s=0.1,
        label=g,
    )
ax.legend()

In [ ]:
import os

if not os.path.exists("tmp"):
    os.makedirs("tmp")
    print(f"Folder 'tmp' created successfully!")
else:
    print(f"Folder 'tmp' already exists.")

adata.write_h5ad("tmp/adata_gated.h5ad")